# [Llm Prompting]

Use LLMs to analyze dataset

* Run on the "**resilient3**" conda env

# Setup

## Library import
We import all the required Python libraries

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [84]:
import os,sys
import joblib
from os import path
import textwrap
from tqdm import tqdm

# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30


# Visualizations
import matplotlib.pyplot as plt

## Local library import
We import all the required local libraries libraries

In [4]:
# Include local library paths
import sys
# add project folders so local libraries can be imported
sys.path.insert(1, os.path.join(os.getcwd()  , '..'))
sys.path.insert(1, os.path.join(os.getcwd()  , '../src'))

In [5]:
# notebook misc functions
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )
    # textwrap.wrap()

def print_wrap(txt, width=120):
    print('\n'.join(textwrap.wrap(txt, width=width)))

def displayAll(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):	
        display(df)  

In [6]:
def load_df(fpath, sep='\t', print_log=True, index_col=0, **kwargs):
    if not index_col is None:
        df = pd.read_csv(fpath, sep=sep, index_col=[index_col], **kwargs)
    else:
        df = pd.read_csv(fpath, sep=sep, **kwargs)
    if print_log:
        print(df.shape)
        print(df.columns)
        display(df.head())
    return df

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [1]:
import openai
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_TOKEN")
os.environ.get("OPENAI_API_KEY")
print(os.environ.get("OPENAI_TOKEN"))
openai.api_key = os.environ.get("OPENAI_TOKEN")

NameError: name 'os' is not defined


# Experiments


In [8]:
df = load_df("../out/paanx_preprocessed.tsv")

(1930, 4)
Index(['text', 'source', 'class', 'tokeep'], dtype='object')


,text,source,class,tokeep
0,i have been dealing with these for quite some ...,Quora,panic,1
1,"i was walking to school, but was running late ...",Quora,panic,1
2,"for me, panic attacks come two ways: randomly ...",Quora,panic,1
3,"i’m driving home, riding a tingly high from sa...",Quora,panic,1
4,it feels like your body turns into a black hol...,Quora,panic,1


In [100]:
# displayAll(df[df["class"] == 'anxiety'].sample(5))
displayAll(df.sample(5))

,text,source,class,tokeep
510,i recommend you talk to a psychiatrist as your anxiety and panic attacks are severely affecting your life. he will probably prescribe klonopin which will be a great help. don't let this disorder ruin your life.,Quora,panic,1
1269,"anxiety is a complex and multifaceted mental health condition that can have many different causes, and often it's difficult to identify a single cause. however, some common factors that can contribute to the development of anxiety include:genetics: research has shown that anxiety can run in families, suggesting that there may be a genetic component to the condition.brain chemistry: imbalances in certain chemicals in the brain, such as serotonin, can contribute to the development of anxiety.trauma: experiencing a traumatic event, such as abuse, violence, or a natural disaster, can lead to the development of anxiety.environmental stressors: chronic stress, financial difficulties, relationship problems, and other stressful life events can trigger or exacerbate anxiety symptoms.personality traits: certain personality traits, such as perfectionism, low self-esteem, and negative thinking patterns, can increase the risk of developing anxiety.medical conditions: certain medical conditions, such as thyroid problems, heart disease, and respiratory disorders, can contribute to the development of anxiety.it's worth noting that anxiety is a complex and individual condition, and what may cause anxiety in one person may not cause anxiety in another. additionally, while identifying the causes of anxiety can be helpful, it's important to remember that anxiety is a treatable condition, and there are a variety of effective treatments available, such as therapy, medication, and lifestyle changes. if you are struggling with anxiety, it's important to reach out to a healthcare professional for help.",Quora,anxiety,1
1473,"dear friend, first of all, i’m really sorry to hear that you're suffering from a mental illness. social anxiety is an extremely common problem and can be incredibly difficult to deal with. i'm sure you've tried many different things without success. i do have some suggestions that may help you. meditate: try sitting meditation, walking meditation, or any other type of meditation you find comfortable. it may help reduce stress, and negative thoughts and improve your overall well-being. talk to someone: it's always helpful to share your feelings with someone who understands and cares about you. they can provide emotional support and advice that may be helpful to you. you can contact heart it out anytime and we are always there to help you. cognitive-behavioral therapy (cbt): this type of therapy can help you change the way you think and feel about social situations. by changing these feelings and thoughts, you can eventually overcome your anxiety symptoms; it involves understanding your brain's relationship with anxiety and how it affects your daily life.",Quora,anxiety,1
1003,"the usual signs that an individual is coming under anxiety attacks, there are several unusual signs also . you should not be mistaken in identifying them and getting quick help for yourself, or someone you know who battles anxiety disorders. so, here are some of them:silence: many could also be surprised on how an individual within the throes of an attack are often silent. but that is actually true. responses to situations vary from one person to another. so does the way a person reacts to anxiety. sudden and grave silence can be a symptom of an anxiety attack.hypersensitivity: an individual facing anxiety attacks are often hypersensitive. it are often to words spoken or to situations happening around them. it also can be to the faintest of noises bringing on utter shock. it also can entail crying call at otherwise not very serious situations.obsessive behavior: while obsessive behavior is the symptom of several mental health issues, it is also a sign of an anxiety attack. obsessing over the location of

## Semantic funcs 

In [9]:
import json
from typing import List
from pydantic import BaseModel, Field
from pydantic.typing import Literal

import openai

from langchain.memory.chat_memory import ChatMessageHistory as ChatMemory
from langchain.memory.chat_memory import ChatMessageHistory


### Narrative analysis 

In [46]:
class NarrativeAnalysisResponse(BaseModel):
    physical_symptoms: List[str] = Field(..., description="Physical symptoms like increased heart rate, sweating, trembling, and dizziness.")
    cognitive_symptoms: List[str] = Field(..., description="Experience of negative thoughts, worry, and fear, and their intensity and duration.")
    negative_behavioral_changes: List[str] = Field(..., description="Negative changes in behavior, such as avoidance of certain situations or activities.")
    positive_behavioral_changes: List[str] = Field(..., description="Strategies or approaches followed to cope with the situation")
    impact_on_daily_life: List[str] = Field(..., description="Mentions of significant impact on a person's daily life, relationships, and overall well-being.")
    stressors: List[str] = Field(..., description="Mentions of the source of stress or fear.")

In [47]:
def post_analysis_narrative(post:str, chat_history:ChatMessageHistory=None, temperature:float = 0.0, model:str='gpt-3.5-turbo'): # 'gpt-4-1106-preview' | 'gpt-3.5-turbo'
    # _prompt = f"""You are an AI psychology agent expert in anxiety and panic attacks. 
    # Analyze the provided social media post and extract the requested information based on the following description:
    # ---------
    # The common ground between panic attacks and anxiety is that both involve the experience of fear and are related to the body's natural response to stress. Additionally, they share some common symptoms, such as:
    # - Physical Symptoms: Both panic attacks and anxiety can manifest with physical symptoms like increased heart rate, sweating, trembling, and dizziness.
    # - Cognitive Symptoms: Individuals with panic attacks and anxiety may experience negative thoughts, worry, and fear, although the intensity and duration may vary.
    # - Behavioral Changes: Both conditions can lead to changes in behavior, such as avoidance of certain situations or activities that trigger anxiety.
    # - Impact on Daily Life: Panic attacks and chronic anxiety can significantly impact a person's daily life, relationships, and overall well-being.
    # While there are similarities, it's important to note that panic attacks are typically more intense and sudden, whereas anxiety is often characterized by a more prolonged and persistent sense of worry or unease.     
    
    # -----
    # Post: 
    # {post}

    # Base your answers only on the provided post do not make up things."""

    _prompt = f"""You are an AI psychology agent expert in anxiety and panic attacks. 
    Analyze the provided social media post and extract:
    - Physical Symptoms: Both panic attacks and anxiety can manifest with physical symptoms like increased heart rate, sweating, trembling, and dizziness.
    - Cognitive Symptoms: Individuals with panic attacks and anxiety may experience negative thoughts, worry, and fear, although the intensity and duration may vary.
    - Negative Behavioral Changes: changes in behavior, such as avoidance of certain situations or activities that trigger anxiety.
    - Positive Behavioral Changes: Strategies or approaches followed to cope with the situation
    - Impact on Daily Life: Panic attacks and chronic anxiety can significantly impact a person's daily life, relationships, and overall well-being.

    Base your answers only on the provided post do not make up things."""    

    #Building a burnout risk profile involves identifying various factors that contribute to burnout and assessing their presence or intensity in an individual's life. 
    # print(f"do_postmortem_analysis --> {chat_history}")
    analysis_extraction_functions = [
        {
            'name': 'narrative_analysis',
            'description': _prompt,
            'parameters': NarrativeAnalysisResponse.schema()
        }
    ]    

    response = openai.ChatCompletion.create(
        model = model,
        temperature=0.0,
        # messages = [], #to_openai_chathistory(chat_history),
        messages = [{'role': 'user', 'content': f"Post: {post}"}],
        functions = analysis_extraction_functions,
        function_call = {'name': 'narrative_analysis'} 
        # function_call = 'auto'
    )

    json_response = json.loads(response['choices'][0]['message']['function_call']['arguments'])    

    return json_response


In [60]:
post_txt = df.iloc[1].text
print_wrap(post_txt)

i was walking to school, but was running late because i had missed my alarm. by the time i made it to the schools corner
the bell had already rung and everybody was in class. i started to run because my first period gym teacher always locked
the door to the locker room after the bell. unfortunately, i was too late. i turned to head outside to where my gym
class would be when two girls who always picked on me started up again. i just couldn’t take the picking at that i
endured everyday and had tried to teach myself to tune out. i tried to make a game of it to give me a laugh. how many
times can these two silly willy’s make me laugh today? but that day i just couldn’t handle it. because it wasn’t a game,
and what they said actually did hurt. the laughs were supposed to help build me up but they were counterproductive. i
sprinted past them and out to an upstairs bathroom close to my next period english class. i felt trapped. not in the
bathroom, not in this world, but in myself. like ever

In [48]:
rs = post_analysis_narrative(post_txt)
rs

{'physical_symptoms': ['increased heart rate',
  'sweating',
  'trembling',
  'dizziness'],
 'cognitive_symptoms': ['negative thoughts', 'worry', 'fear'],
 'negative_behavioral_changes': ['avoidance of certain situations or activities'],
 'positive_behavioral_changes': ['learning new things',
  'taking up yoga or meditation',
  'physical exercise'],
 'impact_on_daily_life': [],
 'stressors': ['social anxiety']}

In [49]:
rs = post_analysis_narrative(post_txt, model="gpt-4-1106-preview")
rs

{'physical_symptoms': [],
 'cognitive_symptoms': [],
 'negative_behavioral_changes': [],
 'positive_behavioral_changes': ['get a friend or family member for support',
  'learn something new every week',
  'take up yoga, meditation, or physical exercise'],
 'impact_on_daily_life': [],
 'stressors': ['social anxiety']}

### Postmortem analysis

In [63]:

class PostmortemAnalysisResponse(BaseModel):
    stressors: List[str] = Field(..., description="mentions of stressors (e.g., conflicts, expectations)")
    coping_mechanisms: List[str] = Field(..., description="coping strategies mentioned by the individual")
    social_environment: List[str] = Field(..., description="details about the social environment, such as demanding, support, and life balance")
    personal_resources: List[str] = Field(..., description="mentions of resilience, self-efficacy, and social support")    
    physical_health: List[str] = Field(..., description="mentions of sleep patterns, exercise, and nutrition")
    emotional_wellbeing: List[str] = Field(..., description="expressions related to mood, emotional exhaustion, satisfaction levels")
    social_identity: List[str] = Field(..., description="information about how the individual perceives his social role and values")
    satisfaction: List[str] = Field(..., description="mentions of satisfaction and fulfillment")
    intervention_strategies: List[str] = Field(..., description="discussions or mentions of strategies to address burnout")



def post_analysis_axis(post:str, chat_history:ChatMessageHistory=None, temperature:float = 0.0, model:str='gpt-3.5-turbo'): # 'gpt-4-1106-preview' | 'gpt-3.5-turbo'
  
    # _prompt = """You are an AI counselor agent expert in burnout. Only based on the chat history, generate "Burnout-Risk-Profile" to evaluate an individual's susceptibility to burnout, which is a state of chronic physical and emotional exhaustion often related to work-related stress. Base your answers only on the user messages."""
    _prompt = """You are an AI psychology agent expert in anxiety and panic attacks. 
    Analyze the provided social media post and extract the requested information. Base your answers only on the mentions personally experienced by the poster."""

    postmortem_analysis_extraction_functions = [
        {
            'name': 'postmortem_analysis',
            'description': _prompt,
            'parameters': PostmortemAnalysisResponse.schema()
        }
    ]    


    response = openai.ChatCompletion.create(
        model = model,
        temperature=0.0,
        # messages = [], #to_openai_chathistory(chat_history),
        messages = [{'role': 'user', 'content': f"Post: {post}"}],
        functions = postmortem_analysis_extraction_functions,
        function_call = {'name': 'postmortem_analysis'} 
        # function_call = 'auto'
    )

    json_response = json.loads(response['choices'][0]['message']['function_call']['arguments'])    

    return json_response


In [64]:
rs = post_analysis_axis(post_txt)
rs

{'stressors': ['running late',
  'being locked out of gym class',
  'being picked on',
  'feeling trapped',
  'hyperventilating',
  'having panic attacks'],
 'coping_mechanisms': ['trying to tune out',
  'making a game of it',
  'laughter'],
 'social_environment': ['girls picking on me'],
 'personal_resources': ['none mentioned'],
 'physical_health': ['hyperventilating', 'feeling trapped', 'panic attacks'],
 'emotional_wellbeing': ['tears falling',
  'feeling terrible',
  'losing control'],
 'social_identity': ['none mentioned'],
 'satisfaction': ['feeling terrible'],
 'intervention_strategies': ['none mentioned']}

In [65]:
rs = post_analysis_axis(post_txt, model="gpt-4-1106-preview")
rs

{'stressors': ['running late',
  'missed alarm',
  'locked door at gym class',
  'bullying from two girls'],
 'coping_mechanisms': ['trying to tune out bullying',
  'making a game of the bullying to laugh'],
 'social_environment': ['bullying', 'lack of support during the incident'],
 'personal_resources': [],
 'physical_health': ['hyperventilation',
  'tears',
  'knees buckling',
  'shaking',
  'heaving chest'],
 'emotional_wellbeing': ['feeling trapped',
  'emotional pain from bullying',
  'feeling overwhelmed',
  'loss of control'],
 'social_identity': [],
 'satisfaction': [],
 'intervention_strategies': []}

### Combined

In [67]:
from functools import partial

In [66]:
def semantic_function(prompt:str, response_schema:BaseModel, messages:[], temperature:float = 0.0, model:str='gpt-3.5-turbo'): # 'gpt-4-1106-preview' | 'gpt-3.5-turbo'
  
    _functions = [
        {
            'name': 'func_extract',
            'description': prompt,
            'parameters': response_schema.schema()
        }
    ]    

    response = openai.ChatCompletion.create(
        model = model,
        temperature=0.0,
        messages = messages,
        functions = _functions,
        function_call = {'name': 'func_extract'} 
    )

    json_response = json.loads(response['choices'][0]['message']['function_call']['arguments'])    

    return json_response

In [131]:

class PostAnalysisResponse(BaseModel):
    anxiety_or_panic: Literal["anxiety", "panic_attack", "none"] = Field(..., description="Does the poster experienced anxiety, a panic attack or none?")
    type_of_post: Literal["Personal_experience", "Giving_information", "Advocacy_and_awareness", "Support_and_encouragement", "Tips_and_strategies", "Question_and_discussion", "Other"] = Field(..., description="Type of the post")
    profesionally_diagnosed: Literal["YES", "NO", "N/A"] = Field(..., description="Does the anxiety or panic attacks have been diagnosed by a profesional?")
    profesionally_treated: Literal["YES", "NO", "N/A"] = Field(..., description="Is the poster being treated by a mental health professional?")
    taking_medication: Literal["YES", "NO", "N/A"] = Field(..., description="Is the poster taking medication for his mental disorder?")
    other_medical_conditions: List[str] = Field(..., description="Other mentions of medical conditions reported by the poster.")
    
    relationship_context: Literal["Co-workers", "Classmates", "Family", "Friends", "Romantic_partners", "Neighbors", "Online_connections", "Other"] = Field(..., description="relationship contexts associated with the reported anxiety or panic event.")
    trigger: Literal["Financial_stress", "Argument_or_conflict", "Traumatic_experience", "Social_event", "Life_transition", "Upcoming_tests", "Health_concern", "Other"] = Field(..., description="Mentioned trigger of the anxiety or panic event.")
    location: Literal["School", "Work", "Home", "Hospital", "Public_spaces", "Club_or_social_venues", "Other"] = Field(..., description="Mentioned location where the anxiety or panic event occurred.")
    event_frequency: Literal["First_time", "Few_times", "Recurrent", "None"] = Field(..., description="Mentioned frequency of the anxiety or panic events.")
    # first_time, is recurrent
    duration_time_scale:Literal["Minutes", "Hours", "Days", "Months", "None"] = Field(..., description="Scale of the duration of the mentioned symptoms ")
    
    stressors: List[str] = Field(..., description="mentions of stressors (e.g., conflicts, expectations)")
    perceived_signs_of_distress: List[str] = Field(..., description="Signs of distress that an external person could use to identify that a person is having a pannic or anxiety attack")
    coping_mechanisms: List[str] = Field(..., description="Coping strategies mentioned by the individual.")
    social_environment: List[str] = Field(..., description="Details about the social environment, such as demanding, support, and life balance")
    physical_symptoms: List[str] = Field(..., description="Mentions of physical symptoms like increased heart rate, sweating, trembling, and dizziness.")
    cognitive_symptoms: List[str] = Field(..., description="Experience of negative thoughts, worry, and fear, and their intensity and duration.")
    emotional_wellbeing: List[str] = Field(..., description="expressions related to mood, emotional exhaustion, satisfaction levels")
    intensity: str = Field(..., description="Intensity level of the mentioned symptoms [Light, Moderate, Strong]")
    duration: List[str] = Field(..., description="Duration of the mentioned symptoms ")
    
    negative_behavioral_changes: List[str] = Field(..., description="Negative changes in behavior mentioned by the individual, such as avoidance of certain -situations or activities.")
    impact_on_daily_life: List[str] = Field(..., description="Mentions of consequences on the individual's daily life, relationships, and overall well-being.")


def post_analysis_combined(post:str, model:str='gpt-3.5-turbo'): # 'gpt-4-1106-preview' | 'gpt-3.5-turbo'
  
    _prompt = """You are an AI psychology agent expert in anxiety and panic attacks. 
    Analyze the provided social media post and extract the requested information. 
    DO NOT MAKE UP INFORMATION NOT MENTIONED IN THE POST."""

    return semantic_function(_prompt, PostAnalysisResponse, model=model, messages = [{'role': 'user', 'content': f"Post: {post}"}])


In [130]:
post_txt = df.iloc[510].text
print_wrap(post_txt)
rs = post_analysis_combined(post_txt)
rs

i recommend you talk to a psychiatrist as your anxiety and panic attacks are severely affecting your life. he will
probably prescribe klonopin which will be a great help. don't let this disorder ruin your life.


{'anxiety_or_panic': 'none',
 'type_of_post': 'Giving_information',
 'profesionally_diagnosed': 'NO',
 'profesionally_treated': 'NO',
 'taking_medication': 'NO',
 'other_medical_conditions': [],
 'relationship_context': 'Other',
 'trigger': 'Other',
 'location': 'Other',
 'event_frequency': 'None',
 'duration_time_scale': 'Other',
 'stressors': [],
 'perceived_signs_of_distress': [],
 'coping_mechanisms': [],
 'social_environment': [],
 'physical_symptoms': [],
 'cognitive_symptoms': [],
 'emotional_wellbeing': [],
 'intensity': 'N/A',
 'duration': [],
 'negative_behavioral_changes': [],
 'impact_on_daily_life': []}

In [168]:
def do_analysis(df, tgt_path, dump_every=100):
    print(f"Doing analysis of: {df.shape}")
    dftmp = []
    j = 0
    for idx, posti in tqdm(df.iterrows(), total=len(df)):
        _rsi = {}
        _rsi["idx"] = idx
        _rsi["post"] = df.at[idx, "text"]
        try:
            _rsx = json.dumps ( post_analysis_combined(_rsi["post"]) )
        except Exception as err:
            _rsx = None

        _rsi["gpt35_response"] = _rsx
        dftmp.append(_rsi)
        j+=1
        if j%dump_every==0:
            pd.DataFrame.from_records( dftmp).to_csv(f"{tgt_path}_{j}.tsv", sep="\t")


    dftmp = pd.DataFrame.from_records( dftmp)
    dftmp.to_csv(f"{tgt_path}_final.tsv", sep="\t")
    return dftmp

# displayAll
(do_analysis(df, tgt_path="../out/post_analysis", dump_every=50))

Doing analysis of: (1930, 4)


100%|██████████| 1930/1930 [2:02:51<00:00,  3.82s/it]  


,idx,post,gpt35_response
0,0,i have been dealing with these for quite some ...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1,1,"i was walking to school, but was running late ...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
2,2,"for me, panic attacks come two ways: randomly ...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
3,3,"i’m driving home, riding a tingly high from sa...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
4,4,it feels like your body turns into a black hol...,"{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
...,...,...,...
1925,1925,i decided not to take the prescription medicat...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1926,1926,speaking to your doctor will help but the most...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1927,1927,lorazepam. i only take it as needed (sparingly...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1928,1928,hi there. block your social media for a days a...,"{""anxiety_or_panic"": ""anxiety"", ""type_of_post""..."


In [170]:
dfx = pd.read_csv("../out/post_analysis_final.tsv", sep='\t', index_col=0)
dfx

,idx,post,gpt35_response
0,0,i have been dealing with these for quite some ...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1,1,"i was walking to school, but was running late ...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
2,2,"for me, panic attacks come two ways: randomly ...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
3,3,"i’m driving home, riding a tingly high from sa...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
4,4,it feels like your body turns into a black hol...,"{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
...,...,...,...
1925,1925,i decided not to take the prescription medicat...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1926,1926,speaking to your doctor will help but the most...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1927,1927,lorazepam. i only take it as needed (sparingly...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1928,1928,hi there. block your social media for a days a...,"{""anxiety_or_panic"": ""anxiety"", ""type_of_post""..."


In [200]:
dfx.loc[dfx[dfx.gpt35_response.isna()].index, "gpt35_response"] = "{}"

In [202]:
dfx

,idx,post,gpt35_response
0,0,i have been dealing with these for quite some ...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1,1,"i was walking to school, but was running late ...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
2,2,"for me, panic attacks come two ways: randomly ...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
3,3,"i’m driving home, riding a tingly high from sa...","{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
4,4,it feels like your body turns into a black hol...,"{""anxiety_or_panic"": ""panic_attack"", ""type_of_..."
...,...,...,...
1925,1925,i decided not to take the prescription medicat...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1926,1926,speaking to your doctor will help but the most...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1927,1927,lorazepam. i only take it as needed (sparingly...,"{""anxiety_or_panic"": ""none"", ""type_of_post"": ""..."
1928,1928,hi there. block your social media for a days a...,"{""anxiety_or_panic"": ""anxiety"", ""type_of_post""..."


In [201]:
dfx[dfx.gpt35_response.isna()]

,idx,post,gpt35_response


In [204]:
dfx = pd.DataFrame.from_records( dfx.gpt35_response.map(lambda v: json.loads(v)) )
dfx

,anxiety_or_panic,type_of_post,profesionally_diagnosed,profesionally_treated,taking_medication,other_medical_conditions,relationship_context,trigger,location,event_frequency,duration_time_scale,stressors,perceived_signs_of_distress,coping_mechanisms,social_environment,physical_symptoms,cognitive_symptoms,emotional_wellbeing,intensity,duration,negative_behavioral_changes,impact_on_daily_life
0,none,Personal_experience,N/A,N/A,N/A,[],Other,Other,Other,None,None,[],[],[],[],[],[],[],None,[],[],[]
1,panic_attack,Personal_experience,NO,NO,NO,[],Classmates,Bullying,School,Recurrent,Minutes,"[Being late for school, Locking of the locker ...","[Running late, Feeling trapped, Hyperventilati...","[Trying to tune out the bullying, Making a gam...","[Picking on me, Feeling alone]","[Hyperventilation, Tears, Knees buckling, Worl...","[Feeling trapped, Losing control, Feeling like...","[Feeling hurt, Feeling trapped, Feeling out of...",Strong,[Over an hour],"[Avoiding the locker room, Staying in the bath...","[Missed class, Feeling terrible]"
2,panic_attack,Personal_experience,N/A,N/A,N/A,[],Other,Other,Public_spaces,Recurrent,Minutes,"[Feeling threatened or unsafe, Worrying too much]","[Feeling out of breath, Dizziness, Tingling in...","[Calming breathing, Clearing the mind]",[],"[Increased heart rate, Chest tightness, Trembl...","[Worrying about dying, Irrational fears, Worry...","[Exhaustion, Confusion, Sleepiness]",Strong,"[Minutes, Seconds]",[Sleeping],[Feeling off for a few days after]
3,panic_attack,Personal_experience,NO,NO,NO,[],Romantic_partners,Darkness_and_deer,Driving,Recurrent,Minutes,"[Darkness, Deer]","[Rapid heartbeat, Difficulty breathing, Tight ...","[Pulling over, Calling mom, Talking to mom]",[],"[Rapid heartbeat, Difficulty breathing, Tight ...","[Fear of hitting a deer, Fear of dying, Fear o...","[Terror, Agony, Relief]",Strong,[10 minutes],[],"[Interrupted driving, Emotional distress]"
4,panic_attack,Personal_experience,NO,NO,NO,[],Other,Other,Other,Recurrent,Seconds,[],[],[],[],"[increased heart rate, sweating, trembling, di...","[losing consciousness, confusion, disorientation]","[anxiety, helplessness, fear]",Strong,[Seconds],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1925,none,Personal_experience,N/A,N/A,NO,[],N/A,N/A,N/A,None,None,[],[],"[taking gaba, l-theanine, and magnesium combin...",[],[],[],"[calm, able to handle things better]",None,[],[],[]
1926,none,Personal_experience,N/A,N/A,N/A,[],Other,Other,Other,None,None,[],[],"[cbt, exposure therapy, mindfulness practices]",[],[],[],[],None,[],[],[]
1927,none,Personal_experience,N/A,N/A,YES,[],Other,Other,Other,None,None,[],[],[],[],[],[],[],None,[],[],[]
1928,anxiety,Personal_experience,NO,NO,NO,[],Online_connections,None,Home,Recurrent,Days,[],[],[],[],[],[],[],None,[],[],[]


In [206]:
dfx.columns

Index(['anxiety_or_panic', 'type_of_post', 'profesionally_diagnosed',
       'profesionally_treated', 'taking_medication',
       'other_medical_conditions', 'relationship_context', 'trigger',
       'location', 'event_frequency', 'duration_time_scale', 'stressors',
       'perceived_signs_of_distress', 'coping_mechanisms',
       'social_environment', 'physical_symptoms', 'cognitive_symptoms',
       'emotional_wellbeing', 'intensity', 'duration',
       'negative_behavioral_changes', 'impact_on_daily_life'],
      dtype='object')

In [212]:
dfx.negative_behavioral_changes.value_counts()[:20]

negative_behavioral_changes
[]                                                                                                                                        1710
[avoidance of certain situations or activities]                                                                                             10
[avoidance behaviors]                                                                                                                        5
[None]                                                                                                                                       3
[Avoidance of certain situations]                                                                                                            2
[Avoidance of social situations]                                                                                                             2
[Avoiding social situations]                                                                                      

In [211]:
for c in ['anxiety_or_panic', 'type_of_post', 'profesionally_diagnosed',
       'profesionally_treated', 'taking_medication',
       'other_medical_conditions', 'relationship_context', 'trigger',
       'location', 'event_frequency', 'duration_time_scale']:
    
    print(f"============= {c} ==================")

    displayAll(dfx[c].value_counts()[:10])a

============= anxiety_or_panic ==================


anxiety_or_panic
anxiety         925
none            506
panic_attack    492
both              1
Name: count, dtype: int64

============= type_of_post ==================


type_of_post
Personal_experience          1024
Giving_information            477
Tips_and_strategies           183
Support_and_encouragement     150
Other                          59
Question_and_discussion        17
Advocacy_and_awareness         14
Name: count, dtype: int64

============= profesionally_diagnosed ==================


...

location
Other            1125
N/A               288
None              158
Home              146
School             51
Work               46
Public_spaces      41
Unknown            11
Hospital            8
Car                 7
Name: count, dtype: int64

============= event_frequency ==================


event_frequency
None          1058
Recurrent      739
N/A             93
First_time      31
Few_times        2
Occasional       1
Name: count, dtype: int64

============= duration_time_scale ==================


duration_time_scale
None       1315
Minutes     386
N/A          93
Days         37
Hours        35
Months       33
Years        18
Weeks         6
Seconds       1
Name: count, dtype: int64